# Extract sources from catalogs  in  OGA

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/18
- update : 2024/02/24


         DatasetType('isolated_star_cat', {instrument, skymap, tract}, DataFrame),
         DatasetType('isolated_star_sources', {instrument, skymap, tract}, DataFrame),
         DatasetType('isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),


In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_07/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_07/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_07/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+34187316c4 	current w_2024_07 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

In [8]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [9]:
DATE = 202402
#filesource_out="finalized_src_table_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [10]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [11]:
#file_out="sourceTable_202312.csv"

## Give the collection

In [12]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

## Config

# Sources

In [13]:
# src : a Source catalog
# srcMatchFull
# finalized_src_table : sape of a pandas dataframe

#'isolated_star_cat', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources', {instrument, skymap, tract}, DataFrame),
#'isolated_star_sources_atlas_refcat2_20220201_match', {skymap, tract}, DataFrame),

#data_product = "isolated_star_sources" 
data_product = "isolated_star_cat" 
#data_product = 'isolated_star_sources_atlas_refcat2_20220201_match' # Nothing
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")
filesource_out=f"{data_product}_{DATE}.csv"

In [14]:
all_df = []

for i, ref in enumerate(datasetRefs):
    print(f"========================datasetType = {data_product} ============================================")
    print("fullId.............................:",ref.dataId.full)
    print("tract..............................:",ref.dataId['tract'])
    tract = ref.dataId['tract']
    df = butler.get(ref)

    if(len(df)>0):
        print(type(df),len(df))
        df['tract'] = tract
        all_df.append(df)
    

========================datasetType = gaia_dr2_20200414 ============================================
fullId.............................: {htm7: 131072}


/tmp/ipykernel_35624/1033383477.py:5: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId.............................:",ref.dataId.full)


KeyError: 'tract'

In [ ]:
df = pd.concat(all_df)

In [ ]:
df

In [ ]:
df.to_csv(filesource_out)

In [ ]:
print(df.columns.tolist())

In [ ]:
df['tract'].unique()

In [ ]:
ref.to_simple()